In [ ]:
import numpy as np
import keras
import keras.layers as layers
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection._split import train_test_split

In [ ]:
#x = [1,    2,        3,           4, 5,                  6, 100]
#y = [0.99, 2.000001, 3.020000011, 4, 4.9999999999999923, 6, 10]

# Just checking out Y = mX + c (m = 1, c = 0 => Y = X)
x = range(0, 100, 1)
y = range(0, 100, 1)

In [ ]:
fig, ax = plt.subplots()
ax.scatter(x, y, edgecolors=(0, 0, 0))
ax.set_xlabel('X')
ax.set_ylabel('Y')
plt.show()

In [ ]:
model = keras.Sequential([
      layers.Dense(1, use_bias=True, input_shape=(1,))
    ])

In [ ]:
optimizer = tf.keras.optimizers.Adam(
    learning_rate=0.01, beta_1=0.9, beta_2=0.99, epsilon=1e-05, amsgrad=False,
    name='Adam')

In [ ]:
model.compile(loss='mse', optimizer=optimizer, metrics=['mae','mse'])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.33)

In [ ]:
# A mechanism that stops training if the validation loss is not improving for more than n_idle_epochs.
n_idle_epochs = 100
earlyStopping = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=n_idle_epochs, min_delta=0.01)
# Creating a custom callback to print the log after a certain number of epochs
class NEPOCHLogger(tf.keras.callbacks.Callback):
    def __init__(self,per_epoch=100):
        '''
        display: Number of batches to wait before outputting loss
        '''
        self.seen = 0
        self.per_epoch = per_epoch
    
    def on_epoch_end(self, epoch, logs=None):
      if epoch % self.per_epoch == 0:
        print('Epoch {}, loss {:.2f}, mae {:.2f}, mse {:.2f}'\
              .format(epoch, logs['loss'], logs['mae'], logs['mse']))

log_display = NEPOCHLogger(per_epoch=10)

In [ ]:
# NEED MLOPS STUFF HERE FOR HYPERPARAMS AND RESULTS

In [ ]:
n_epochs = 2000
history = model.fit(
  X_train, y_train, batch_size=32,
  epochs=n_epochs, verbose=0, callbacks=[earlyStopping, log_display])

In [ ]:
predictions = model.predict(X_test).flatten()

In [ ]:
a = plt.axes(aspect='equal')
plt.scatter(predictions, y_test, edgecolors=(0, 0, 0))
plt.xlabel('True Values')
plt.ylabel('Predictions')
lims = [0, 10]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims, lims)

In [ ]:
model.predict([10,20,30,200])

In [ ]:
#model.save("linear.h5") # for HDF5 format

# For converting to ONNX
tf.saved_model.save(model, "./models/saved_models/linear_model")


In [ ]:
print("===============================================================================================================================================")
print("\n\n>>> IMPORTANT: CONVERT TO ONNX MODEL FORMAT\n\n$ python3 -m tf2onnx.convert --saved-model 'models/saved_models/linear_model' --output 'models/onnx_models/linear.onnx'\n\n")
print("===============================================================================================================================================")

In [ ]:
import onnxruntime
session = onnxruntime.InferenceSession("./models/onnx_models/linear.onnx", providers=['CPUExecutionProvider'])


xrange = np.arange(1000, 1025, 1)
x_live = np.array([xrange[i:i+1] for i in range(0,len(xrange),1)]).astype(np.float32)
print(x_live)

ortvalue = onnxruntime.OrtValue.ortvalue_from_numpy(x_live)
ortvalue.device_name()  # 'cpu'
ortvalue.shape()        # shape of the numpy array X
ortvalue.data_type()    # 'tensor(float)'
ortvalue.is_tensor()    # 'True'

input_name = session.get_inputs()[0].name
print(input_name)

output_name = session.get_outputs()[0].name
print(output_name)

np.array_equal(ortvalue.numpy(), x_live)  # 'True'



In [ ]:
results = session.run(None, {input_name: ortvalue})
print(results)

## Running model inferencing on Triton model server

`$ docker pull nvcr.io/nvidia/tritonserver:22.03-py3`

`$ docker run --rm -p8000:8000 -p8001:8001 -p8002:8002 -v $(pwd)/triton_model_repository:/models nvcr.io/nvidia/tritonserver:22.03-py3 tritonserver --model-repository=/models`

In [ ]:
import numpy as np
import tritonclient.http as httpclient

In [ ]:
input0_data = np.array([[1231231]], dtype=np.float32)
#input0_data = input0_data.reshape(-1,1)

In [ ]:
inputs = [httpclient.InferInput('dense_input', [1, 1], "FP32")]
inputs[0].set_data_from_numpy(input0_data, binary_data=False)

outputs = []
outputs.append(httpclient.InferRequestedOutput('dense', binary_data=False))

In [ ]:
triton_client = httpclient.InferenceServerClient(url="0.0.0.0:8000")

results = triton_client.infer("linear_model_onnx", inputs, outputs=outputs)

print(results)

In [ ]:
results.get_response()["outputs"][0]["data"]